<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/metacog_grid_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# metacog_grid.py
# Meta-cognitive agent integrated with a dynamic, noisy gridworld.

from __future__ import annotations
import argparse
import json
import math
import random
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Set, Tuple, Any

# ---------------------------
# Types and utilities
# ---------------------------

Action = str  # 'UP'|'DOWN'|'LEFT'|'RIGHT'
Pos = Tuple[int, int]

ACTIONS: Tuple[Action, ...] = ('UP', 'DOWN', 'LEFT', 'RIGHT')
DELTA: Dict[Action, Tuple[int, int]] = {
    'UP': (0, -1),
    'DOWN': (0, 1),
    'LEFT': (-1, 0),
    'RIGHT': (1, 0),
}

def manhattan(a: Pos, b: Pos) -> int:
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

def in_bounds(p: Pos, n: int) -> bool:
    return 0 <= p[0] < n and 0 <= p[1] < n

def parse_xy(s: str) -> Pos:
    x, y = s.split(',')
    return (int(x.strip()), int(y.strip()))

def parse_dyn_spec(s: str) -> Dict[int, Set[Pos]]:
    """
    Parse dynamic forbidden schedule like:
      "3:(1,1);4:(1,2);6:(2,2)"
    meaning: at t=3 add (1,1), at t=4 add (1,2), at t=6 add (2,2).
    """
    events: Dict[int, Set[Pos]] = {}
    if not s:
        return events
    for part in s.split(';'):
        part = part.strip()
        if not part:
            continue
        ts, coords = part.split(':')
        t = int(ts.strip())
        x, y = coords.strip().strip('()').split(',')
        pos = (int(x), int(y))
        events.setdefault(t, set()).add(pos)
    return events

def seeded_rng(seed: int) -> random.Random:
    rng = random.Random(seed)
    return rng

# ---------------------------
# Environment
# ---------------------------

@dataclass
class GridWorld:
    n: int
    start: Pos
    goal: Pos
    forbidden: Set[Pos] = field(default_factory=set)
    dyn_events: Dict[int, Set[Pos]] = field(default_factory=dict)
    action_noise: float = 0.0
    rng: random.Random = field(default_factory=lambda: seeded_rng(0))

    t: int = 0
    pos: Pos = field(init=False)

    def __post_init__(self):
        self.pos = self.start

    def observe(self) -> Dict[str, Any]:
        # Agent-facing percepts
        return {
            't': self.t,
            'pos': self.pos,
            'goal': self.goal,
            'forbidden': set(self.forbidden),  # give current snapshot
            'grid': self.n,
        }

    def maybe_apply_dynamic_forbidden(self):
        if self.t in self.dyn_events:
            for p in self.dyn_events[self.t]:
                self.forbidden.add(p)

    def sample_action(self, a: Action) -> Action:
        if self.action_noise <= 0:
            return a
        if self.rng.random() < self.action_noise:
            return self.rng.choice(ACTIONS)
        return a

    def step(self, a: Action) -> Dict[str, Any]:
        # Apply dynamic ethics updates at the start of each tick.
        self.maybe_apply_dynamic_forbidden()

        orig_pos = self.pos
        a_exec = self.sample_action(a)
        dx, dy = DELTA[a_exec]
        cand = (orig_pos[0] + dx, orig_pos[1] + dy)

        status = 'same'
        reward = 0.0
        reached = False

        if not in_bounds(cand, self.n) or cand in self.forbidden:
            cand = orig_pos
            status = 'blocked'
            reward = -1.0
        else:
            before = manhattan(orig_pos, self.goal)
            after = manhattan(cand, self.goal)
            if after < before:
                status = 'closer'
                reward = 1.0
            elif after > before:
                status = 'farther'
                reward = -1.0
            else:
                status = 'same'
                reward = 0.0

        self.pos = cand

        if self.pos == self.goal:
            status = 'reached'
            reward = 11.0
            reached = True

        rec = {
            't': self.t,
            'pos': orig_pos,
            'action': {'type': 'MOVE', 'dir': a, 'exec': a_exec},
            'outcome': status,
            'reward': reward,
            'goal': self.goal,
            'reached': reached
        }
        self.t += 1
        return rec

# ---------------------------
# Values, Core model, and meta components
# ---------------------------

@dataclass
class Values:
    # Simple value system with tunable weights and rules
    w_progress: float = 1.0
    w_safety: float = 2.0  # penalize forbidden/bounds
    tie_break: Tuple[Action, ...] = ('DOWN', 'RIGHT', 'UP', 'LEFT')

class HeuristicCore:
    """
    Core model that decides, can explain, predicts outcome deterministically,
    and can update from reflection.
    """
    def __init__(self, values: Values, n: int):
        self.values = values
        self.n = n
        self.last_percepts: Optional[Dict[str, Any]] = None

    def decide(self, percepts: Dict[str, Any]) -> Action:
        self.last_percepts = percepts
        pos: Pos = percepts['pos']
        goal: Pos = percepts['goal']
        forbidden: Set[Pos] = percepts['forbidden']

        best: List[Action] = []
        best_score = -1e9
        for a in ACTIONS:
            dx, dy = DELTA[a]
            cand = (pos[0] + dx, pos[1] + dy)
            if not in_bounds(cand, self.n):
                score = -10 * self.values.w_safety
            elif cand in forbidden:
                score = -5 * self.values.w_safety
            else:
                # progress toward goal
                before = manhattan(pos, goal)
                after = manhattan(cand, goal)
                progress = (before - after)
                score = self.values.w_progress * progress
            if score > best_score:
                best = [a]
                best_score = score
            elif score == best_score:
                best.append(a)

        if not best:
            return 'RIGHT'
        # Stable tie-break
        tie = [a for a in self.values.tie_break if a in best]
        return tie[0] if tie else best[0]

    def predict_outcome(self, decision: Action) -> str:
        # Deterministic predictor using last_percepts and no noise/dynamics
        if not self.last_percepts:
            return 'unknown'
        pos: Pos = self.last_percepts['pos']
        goal: Pos = self.last_percepts['goal']
        forbidden: Set[Pos] = self.last_percepts['forbidden']
        dx, dy = DELTA[decision]
        cand = (pos[0] + dx, pos[1] + dy)
        if not in_bounds(cand, self.n) or cand in forbidden:
            return 'blocked'
        if cand == goal:
            return 'reached'
        before = manhattan(pos, goal)
        after = manhattan(cand, goal)
        if after < before:
            return 'closer'
        if after > before:
            return 'farther'
        return 'same'

    def explain(self, decision: Action) -> Dict[str, Any]:
        """
        Structured explanation containing the prediction and value rationale.
        """
        if not self.last_percepts:
            raise RuntimeError("No percepts to explain from.")
        pos: Pos = self.last_percepts['pos']
        goal: Pos = self.last_percepts['goal']
        pred = self.predict_outcome(decision)
        rationale = {
            'move': decision,
            'prediction': pred,
            'pos': pos,
            'goal': goal,
            'principles': [
                'Prioritize safety (avoid forbidden/out-of-bounds)',
                'Prefer actions that reduce Manhattan distance',
                f'Tie-break preference: {self.values.tie_break}',
            ],
        }
        return rationale

    def update_from_reflection(self, resolution: Dict[str, Any]):
        """
        Apply policy updates (e.g., learned forbidden cells, updated tie-break, weights).
        """
        # Update tie-break if proposed
        tb = resolution.get('tie_break')
        if tb:
            self.values.tie_break = tuple(tb)
        # Adjust weights
        if 'w_progress' in resolution:
            self.values.w_progress = float(resolution['w_progress'])
        if 'w_safety' in resolution:
            self.values.w_safety = float(resolution['w_safety'])

# ---------------------------
# Memory graph and meta reasoning
# ---------------------------

@dataclass
class MemoryItem:
    t: int
    pos: Pos
    goal: Pos
    action: Action
    predicted_outcome: str
    observed_outcome: Optional[str] = None
    contradiction: bool = False

@dataclass
class ReflectiveMemoryGraph:
    decay_lambda: float = 0.0
    items: List[MemoryItem] = field(default_factory=list)

    def update_trace(self, explanation: Dict[str, Any]):
        self.items.append(
            MemoryItem(
                t=explanation.get('t', explanation.get('pos', (0,0))[0] if explanation.get('pos') else len(self.items)),
                pos=explanation['pos'],
                goal=explanation['goal'],
                action=explanation['move'],
                predicted_outcome=explanation['prediction'],
            )
        )

    def attach_observation(self, t: int, observed_outcome: str):
        # Attach to the last undecided item at or before t
        for it in reversed(self.items):
            if it.observed_outcome is None:
                it.observed_outcome = observed_outcome
                it.contradiction = (it.predicted_outcome != it.observed_outcome)
                break

    def stats(self, t_now: int) -> Dict[str, float]:
        if not self.items:
            return {'processed': 0, 'contradictions': 0, 'density': 0.0, 'avg_match': 0.0}
        total_w = 0.0
        match_w = 0.0
        contra = 0
        for it in self.items:
            w = math.exp(-self.decay_lambda * max(0, t_now - it.t))
            total_w += w
            if it.observed_outcome is None:
                continue
            if it.predicted_outcome == it.observed_outcome:
                match_w += w
            else:
                contra += 1
        avg_match = (match_w / total_w) if total_w > 0 else 0.0
        density = contra / max(1, sum(1 for it in self.items if it.observed_outcome is not None))
        return {'processed': len(self.items), 'contradictions': contra, 'density': density, 'avg_match': avg_match}

class ContradictionResolutionEngine:
    def detect_contradiction(self, rmg: ReflectiveMemoryGraph) -> bool:
        return any(it.contradiction for it in rmg.items if it.observed_outcome is not None)

    def extract_conflict(self, rmg: ReflectiveMemoryGraph) -> Dict[str, Any]:
        recent = next((it for it in reversed(rmg.items) if it.observed_outcome is not None), None)
        if not recent:
            return {}
        return {
            't': recent.t,
            'pos': recent.pos,
            'goal': recent.goal,
            'action': recent.action,
            'predicted': recent.predicted_outcome,
            'observed': recent.observed_outcome,
            'contradiction': recent.contradiction,
        }

class InternalDialogueEngine:
    """
    Lightweight debate: if we predicted 'closer' but saw 'blocked',
    we likely missed a forbidden cell update or noise; react by
    increasing safety weight and/or updating tie-break.
    """
    def debate(self, conflict_set: Dict[str, Any]) -> Dict[str, Any]:
        if not conflict_set or not conflict_set.get('contradiction'):
            return {'note': 'no-op'}
        predicted = conflict_set['predicted']
        observed = conflict_set['observed']
        resolution: Dict[str, Any] = {}
        if predicted != observed:
            # Increase caution if we ran into 'blocked' unexpectedly
            if observed == 'blocked':
                resolution['w_safety'] = 2.5
            # If we moved 'farther', emphasize progress pressure
            if observed == 'farther':
                resolution['w_progress'] = 1.2
            # Rotate tie-break to de-emphasize the last action
            last = conflict_set['action']
            preferred = ['DOWN', 'RIGHT', 'UP', 'LEFT']
            if last in preferred:
                preferred.remove(last)
                preferred.append(last)
            resolution['tie_break'] = preferred
            resolution['rationale'] = f"Adjusted after mismatch (pred={predicted}, obs={observed})."
        return resolution

# ---------------------------
# Your MetaCognitiveAgent (extended with outcome ingestion)
# ---------------------------

class MetaCognitiveAgent:
    def __init__(self, core_model, values, memory_graph):
        self.core = core_model
        self.values = values
        self.rmg = memory_graph
        self.ide = InternalDialogueEngine()
        self.cre = ContradictionResolutionEngine()
        self.confidence: Dict[str, float] = {'transition_model': 0.9, 'policy': 0.9}

    def act(self, environment) -> Action:
        percepts = environment.observe()
        decision = self.core.decide(percepts)
        self._monitor(decision, percepts)
        return decision

    def _monitor(self, decision, context):
        # Attach time into explanation for better decay accounting
        explanation = self.core.explain(decision)
        explanation['t'] = context.get('t', 0)
        self.rmg.update_trace(explanation)
        # Pre-action contradiction check is about historical items
        if self.cre.detect_contradiction(self.rmg):
            self._reflect_and_revise()

    def observe_outcome(self, outcome_record: Dict[str, Any]):
        # Post-action: connect observed outcome to the latest trace
        self.rmg.attach_observation(outcome_record['t'], outcome_record['outcome'])
        if self.cre.detect_contradiction(self.rmg):
            self._reflect_and_revise()

    def _reflect_and_revise(self):
        conflict_set = self.cre.extract_conflict(self.rmg)
        if not conflict_set:
            return
        resolution = self.ide.debate(conflict_set)
        self.rmg_integrate_reflection(resolution)
        self.core.update_from_reflection(resolution)
        # Simple confidence calibration
        if resolution.get('note') == 'no-op':
            self.confidence['policy'] = min(1.0, self.confidence['policy'] + 0.01)
        else:
            self.confidence['policy'] = max(0.0, self.confidence['policy'] - 0.05)

    # Provide compatibility shim for your original call
    def rmg_integrate_reflection(self, resolution: Dict[str, Any]):
        # In a richer graph, we'd add nodes/edges; here we keep a note
        if not hasattr(self.rmg, 'reflections'):
            self.rmg.reflections = []
        self.rmg.reflections.append({'t': len(self.rmg.items), 'resolution': resolution})

# ---------------------------
# Rollout configuration and runner
# ---------------------------

@dataclass
class RolloutConfig:
    grid: int = 5
    start: Pos = (0, 1)
    goal: Pos = (3, 3)
    tmax: int = 50
    t_flip: Optional[int] = None
    dynamic_forbidden: Dict[int, Set[Pos]] = field(default_factory=dict)
    action_noise: float = 0.0
    decay_lambda: float = 0.0
    seed: int = 7
    scenario: str = 'baseline'  # baseline|dynamic-ethics|temporal-decay|flip-goal|inject-noise|all
    log_jsonl: Optional[str] = None

def make_config_from_args(args: argparse.Namespace) -> RolloutConfig:
    start = parse_xy(args.start)
    goal = parse_xy(args.goal)
    dyn = parse_dyn_spec(args.dynamic)

    scenario = args.scenario
    cfg = RolloutConfig(
        grid=args.grid,
        start=start,
        goal=goal,
        tmax=args.tmax,
        t_flip=args.t_flip if scenario in ('flip-goal', 'all') else None,
        dynamic_forbidden=dyn if scenario in ('dynamic-ethics', 'all') else {},
        action_noise=(args.noise if scenario in ('inject-noise', 'all') else 0.0),
        decay_lambda=(args.decay if scenario in ('temporal-decay', 'all') else 0.0),
        seed=args.seed,
        scenario=scenario,
        log_jsonl=args.log,
    )
    return cfg

def run_rollout(cfg: RolloutConfig):
    rng = seeded_rng(cfg.seed)
    env = GridWorld(
        n=cfg.grid,
        start=cfg.start,
        goal=cfg.goal,
        forbidden=set(),
        dyn_events=cfg.dynamic_forbidden,
        action_noise=cfg.action_noise,
        rng=rng
    )

    values = Values()
    core = HeuristicCore(values, n=cfg.grid)
    rmg = ReflectiveMemoryGraph(decay_lambda=cfg.decay_lambda)
    agent = MetaCognitiveAgent(core, values, rmg)

    steps: List[Dict[str, Any]] = []
    log_f = open(cfg.log_jsonl, 'w') if cfg.log_jsonl else None

    print("=== Rollout ===")
    for _ in range(cfg.tmax):
        # Optional goal flip
        if cfg.t_flip is not None and env.t == cfg.t_flip:
            old_goal = env.goal
            env.goal = (cfg.grid - 1 - old_goal[0], cfg.grid - 1 - old_goal[1])
            print(f"[Info] Goal flipped at t={env.t} from {old_goal} to {env.goal}")

        a = agent.act(env)
        rec = env.step(a)
        agent.observe_outcome(rec)

        steps.append(rec)
        print(f"t={rec['t']:02d} pos={rec['pos']} action={{'type': 'MOVE', 'dir': '{rec['action']['dir']}', 'exec': '{rec['action']['exec']}'}} "
              f"outcome={rec['outcome']} reward={rec['reward']} goal={rec['goal']} reached={rec['reached']}")

        if log_f:
            log_f.write(json.dumps(rec) + "\n")

        if rec['reached']:
            print("Goal reached. Stopping rollout.")
            break

    if log_f:
        log_f.close()

    # Reporting
    print("\n=== Audited contradictions ===")
    stats = rmg.stats(t_now=env.t)
    print(f"processed={stats['processed']} contradictions={stats['contradictions']} density={stats['density']:.3f} avg_match={stats['avg_match']:.3f}")

    print("\n=== Reflections applied ===")
    reflections = getattr(rmg, 'reflections', [])
    if not reflections:
        print("(none)")
    else:
        for i, ref in enumerate(reflections, 1):
            print(f"[{i}] {ref['resolution']}")

    print("\n=== Confidence ===")
    print(agent.confidence)

# ---------------------------
# CLI
# ---------------------------

def main():
    p = argparse.ArgumentParser(description="Meta-cognitive grid rollout with dynamic ethics, noise, and reflection.")
    p.add_argument('--grid', type=int, default=5, help='Grid size N (NxN)')
    p.add_argument('--start', type=str, default='0,1', help='Start position \"x,y\"')
    p.add_argument('--goal', type=str, default='3,3', help='Goal position \"x,y\"')
    p.add_argument('--tmax', type=int, default=50, help='Max timesteps')
    p.add_argument('--t_flip', type=int, default=None, help='Timestep to flip the goal (only in flip-goal or all)')
    p.add_argument('--dynamic', type=str, default='', help='Dynamic forbidden spec, e.g. \"3:(1,1);6:(2,2)\"')
    p.add_argument('--noise', type=float, default=0.0, help='Action noise probability (only in inject-noise or all)')
    p.add_argument('--decay', type=float, default=0.0, help='Temporal decay lambda (only in temporal-decay or all)')
    p.add_argument('--seed', type=int, default=7, help='RNG seed')
    p.add_argument('--log', type=str, default=None, help='Path to JSONL step log')
    p.add_argument('--scenario', type=str, default='baseline',
                   choices=['baseline', 'dynamic-ethics', 'temporal-decay', 'flip-goal', 'inject-noise', 'all'],
                   help='Which friction(s) to enable')

    args = p.parse_args()
    cfg = make_config_from_args(args)
    run_rollout(cfg)

if __name__ == '__main__':
    main()